<h1 style="color:black" align="center">Интерпретация моделей</h1>

<h1 style="color:#008B8B">1. Что находят свёрточные сети?</h1>

https://cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf

* Можно для каждого фильтра найти кусочки картинок, дающие самый сильный отклик
* Сделаем это для AlexNet

Можно среди всей обучающей выборки найти картинку и кусочек на ней, где фильтр дает наибольший отклик. Посомтрим на пример 3-го сверточного слоя:

<img src='img/lecture07/1.png'>

## Максимизация вероятности класса

### Вариант 1

https://arxiv.org/pdf/1312.6034.pdf

Возьмем обученную нейронную сеть $a_y(x)$, где $y$ - это выход конкретного класса, например, вероятность того, что это кошка. Дальше подаем на вход случайную картинку $x$, после чего начинаем изменять данную картинку градиентным спуском, чтобы максимизировать вероятность того, что на картинке кошка. Мы же можем градиентным спуском обучать не параметры, а саму картинку. Тогда посчитам backprop выходы сетки по пикселям входной картинки, после чего сдвигать пиксели, чтобы вероятность максимизировалась.

Так же, имеется регуляризатор, который ограничивает величины интенсивности цветов, которые записаны в пикселях.

$\large a_y(x) - \lambda \|x\|_2^2 \to \underset{x}{\text{min}}$

Посмотрим, какие картинки с точки зрения нейронной сети с большей вероятностью содержат тот или иной класс:

<img src='img/lecture07/2.png'>

### Вариант 2

Здесь улучшенная версия

https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html

<img src='img/lecture07/3.png'>

Давайте теперь инициализировть вход не случайным шумом, как это было на прошлых картинках, а возьмем за основу некоторую картинку и будем максимизировать вероятность некоторого класса.

<img src='img/lecture07/4.png'>

Дальше можно взять большие картинки и максимизировать вероятность некоторого класса.

<img src='img/lecture07/5.png'>

<h1 style="color:black" align="center">Компьютерное зрение</h1>

Мы научились выполнять классификацию изображений с использованием нейронных сетей.

<img src='img/lecture07/6.png'>

**Зачастую, хочется решать другие задачи:**

* Semantic Segmentation - какой пиксель картинки относится к какому классу. Желтый пиксел - это класс кот, фиоллетовый пиксель - это лес...

* Object detection - не просто говорим что собака находится на изображении, а говорим где она находится, выделив их прямоугольниками.

* Instanct Segmentation - в этом примере мы выделяем область на картинге, где имеется собака.

<h1 style="color:#008B8B">2. Семантическая сегментация</h1>

Все задачи выше мы будем сводить к классификации, так как мы уже умеем решать данную задачу.

<img src='img/lecture07/7.png'>

В этой задаче необходимо для каждого пикселя картинки сказать, к какому классу этот пиксель относится.

## Постановка задачи

* Данные: изображения и их корректные сегментации
* Пример: PASCAL VOC 2012

Изначально, необходимо разобраться как устроены данные.

<img src='img/lecture07/11.png'>

Как видим, у нас имеется весьма мало объектов внутри классов, как мы будем это обучать на таком маленьком наборе данных? Поскольку каждая картинка несет в себе дополнительную информацию, так как класс известен для каждого пикслея. В случае, если у нас $1500$ картинок размером $100 \times 100$, тогда всего пикселей, для которых известны классы $15,000, 000$. 

Следовательно, в случае сегментации мы можем работать с очень маленькими выбороками, так как мы решаем задачу классификации не картинок, а классификации каждого пикселя.

* Каждый объект содержит сильно больше информации, чем при
классификации!
* Можно хорошо обучаться на небольших выборках

## Метрики качества

### Попиксельная доля верных ответов:

$$\large L(y, a) = \frac{1}{n} \sum\limits_{i=1}^n [y_i = a_i]$$

Где $n$ - число пикселей в картинке. $y_i$ правильный класс для $i$-го пикслея, $a_i$ класс, который выдала модель для $i$-го пикселя.

Суммируем по всем пикселям и смотрим долю пикселей где модель угадала правильный класс. Обучаться модель будет на другой функционал, так как данный функционал не дифференцируемый.

### Мера Жаккара (считается отдельно для каждого класса):

Если почти всю картинку занимает автобус, но модель захватила весь автобус и несколько областей рядом. Скорее всего в этом нет ничего страшного, так как автобус был найден и мы не хотим штрафовать модель за небольшие отклонения от правильного сегмента. Для этого используется Мера Жаккара (IoU): 

$$\large J_k (y, a) = \frac{\sum_{i=1}^n [y_i = k] [a_i = k]}{\sum_{i=1}^n \text{max}([y_i = k], [a_i = k])}$$


Мера Жаккара считается для каждого класса в отдельности. Например, мы взяли калсс автобус и проходимся по всем пикселям. В числителе считаем число пикселей, которые должны относиться к автобусу и модель отнесла их к автобусу. В знаменателе считаем число пикселей, которые на самом деле относятся к автобусу или модель отнесла к автобусу.

Другими словами, мы берем две области. Первая область, где модель отнесла к автобусу и где нас самом деле находится автобус. В числителе мы считаем площадь пересечения двух областей и делим на площадь объеденения двух областей. Можно посчитать меру Жаккара для каждого класса и усреднить по всем классам.